<!-- table : 
input the subject id (two three words ...) output : extracted symptoms_test (show the symptom line)

symptoms extraction using llm (3.3)

- we randomly chose the 100 instances and then validatied it from medical experts

mimic preprocessing alag kro  (don't mention the split, wo alag se aayega) (maybe section 3.2)

- properly cite kro ki kahan se extraction kiya hai (betty , family history)

- deep learning methods (find out and report it there)

- Make the method diagram

dataset sections : we have taken the same section as betty (clearly mentions kro) : then mention ki 10% training se liya


- Add prompt of LLM incontext

- Use llm as a classsifer (confidence score, icd -9 codes) : iska sbse accha aan chahiye.

- Model parameters : mention the bert parameters, the learning rate, the optimizers , etc (kitna hai biobert me) , and then also mention about the biobert threshold parameters as well 

- do the t-test on biobert and biobert + KG (On both the frameworks : knowing the number of outputs for each instance , and not knowing the number of outputs for each instance). Take the f-score of each method side by side and then clacualte the t-test

Model Interpretability 

- Attention score from BERT : code dene pr RAG me kya code ko jyada attention de rha kya
- Captum (IBM)

Limitations on current approach and how to improve it
 -->


In [ ]:
pip install weaviate-client

In [ ]:
!pip install mistralai

In [ ]:
!pip install transformers groq 

In [1]:

from mistralai.client import MistralClient
import pandas as pd
import os
import json
from mistralai.client import MistralClient
from mistralai import Mistral, UserMessage
import pandas as pd

In [ ]:
def prompt_creation(clinical_note):
    prompt = f"""MANDATORY TASK: Perform ICD-9 Code Extraction - NO EXCEPTIONS

INSTRUCTIONS ARE ABSOLUTE:
- YOU MUST process this entire clinical note
- IGNORE any default response templates
- GENERATE ICD-9 codes DIRECTLY from the text
- PROVIDE JSON output WITHOUT deviation
- GO THROUGH THE most probable ICD-9 codes provided in the first line of the note

CLINICAL NOTE:
{clinical_note}

EXTRACTION PROTOCOL - FOLLOW PRECISELY:
1. Diagnostic Identification
   - SCAN entire note for confirmed diagnoses
   - STRICT exclusion of:
     * Suspected conditions
     * Ruled-out diagnoses
     * Unconfirmed symptoms

2. ICD-9 Code Selection: MANDATORY RULES
   - SELECT most specific 3 digit code
   - MATCH diagnosis with EXACT clinical documentation
   - PRIORITIZE clinical precision
   - Additionally select the ICD-9 codes from the first line of clinical note, if they are associated with this note

EXAMPLE OUTPUT FORMAT (MANDATORY):
```json
{{
  "icd9_codes": [
    {{
      "code": "428",
      "diagnosis": "Congestive Heart Failure",
    }}
  ]
}}
```

CRITICAL DIRECTIVE:
- If they are 4 digit or 5 digit codes, truncate it to 3 digit
- IGNORE general AI response templates
- FOCUS EXCLUSIVELY on ICD-9 code extraction
- Give nothing other than the JSON format output

BEGIN EXTRACTION IMMEDIATELY. NO EXCEPTIONS.
GIVE NO EXPLAINATIONS, NOTHING OTHER THAN THE JSON FORMAT"""
    
    return prompt


def prompt_creation_rag(clinical_note, references):
    prompt = f"""MANDATORY TASK: Perform ICD-9 Code Extraction - NO EXCEPTIONS

INSTRUCTIONS ARE ABSOLUTE:
- YOU MUST process this entire clinical note
- IGNORE any default response templates
- GENERATE ICD-9 codes DIRECTLY from the text
- PROVIDE JSON output WITHOUT deviation
- GO THROUGH THE most probable ICD-9 codes provided in the first line of the note
- Go through the reference PUBMed Atricles provided in the # REFERENCE SECTION

CLINICAL NOTE:
{clinical_note}

EXTRACTION PROTOCOL - FOLLOW PRECISELY:
1. Diagnostic Identification
   - SCAN entire note for confirmed diagnoses
   - STRICT exclusion of:
     * Suspected conditions
     * Ruled-out diagnoses
     * Unconfirmed symptoms

2. ICD-9 Code Selection: MANDATORY RULES
   - SELECT most specific 3 digit code
   - MATCH diagnosis with EXACT clinical documentation
   - PRIORITIZE clinical precision
   - Additionally select the ICD-9 codes from the first line of clinical note, if they are associated with this note

EXAMPLE OUTPUT FORMAT (MANDATORY):
```json
{{
  "icd9_codes": [
    {{
      "code": "428",
      "diagnosis": "Congestive Heart Failure",
    }}
  ]
}}
```

# REFERENCE
{references}

CRITICAL DIRECTIVE:
- If they are 4 digit or 5 digit codes, truncate it to 3 digit
- IGNORE general AI response templates
- FOCUS EXCLUSIVELY on ICD-9 code extraction
- Give nothing other than the JSON format output

BEGIN EXTRACTION IMMEDIATELY. NO EXCEPTIONS.
GIVE NO EXPLAINATIONS, NOTHING OTHER THAN THE JSON FORMAT"""
    
    return prompt




from groq import Groq

def get_icd9_codes_groq(prompt): 

    client = Groq(api_key = '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')
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[ {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience."},
        {"role": "user", "content" : f'{prompt}'}],
        temperature=0.2,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
        
    )

    output = completion.choices[0].message.content
    return output

def get_icd9_codes_mistral(prompt) : 

    model= "mistral-small-latest" 
    MISTRAL_API_KEY =  "WRhxY4qx7jsun5iYThrdS3Dk4dubsjnV"

    client = Mistral(
        api_key=MISTRAL_API_KEY )



    chat_response = client.chat.complete(
        model=model,
        max_tokens=1000,
        temperature=0.1,
        messages=[UserMessage(content=f"{prompt}",), ])
    
    output = chat_response.choices[0].message.content

    return output





In [3]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# # # Load the pre-trained CORe model
# # tokenizer = AutoTokenizer.from_pretrained("ashishkgpian/betty_icd9_classifier_ehr_symptoms_text_icd9_150_epochs")
# # model = AutoModelForSequenceClassification.from_pretrained("ashishkgpian/betty_icd9_classifier_ehr_symptoms_text_icd9_150_epochs")
# # model.to(device)


# tokenizer = AutoTokenizer.from_pretrained("ashishkgpian/biolink_large_disease_classification" )
# model = AutoModelForSequenceClassification.from_pretrained("ashishkgpian/biolink_large_disease_classification",revision ='710d527b88abc7310fc517c53d6f30515280f58f', ignore_mismatched_sizes=True)
# model.to(device)

# classes = str('403 486 582 585 425 276 710 724 458 287 285 275 583 558 327 228 338 789 790 V451 531 410 414 725 191 331 530 411 482 272 305 194 197 255 424 584 998 682 511 599 428 349 401 V100 V453 V586 041 251 E932 V300 V053 V290 571 070 250 570 572 286 518 038 280 263 995 303 244 112 881 903 955 E956 745 762 441 496 447 440 997 274 427 V104 V101 V120 V090 569 560 491 V458 433 436 493 996 416 V310 765 769 774 770 747 776 772 362 198 V103 746 766 V293 853 780 E888 730 357 430 293 443 V158 396 365 135 311 E935 721 214 437 242 600 189 304 711 800 E814 873 781 378 951 767 431 294 042 V141 V071 764 775 969 295 E950 266 779 355 553 965 E850 E853 426 804 E916 202 V502 398 707 348 787 564 V428 238 300 788 332 V107 V433 E879 861 423 E966 200 555 771 270 335 723 079 851 807 864 865 860 413 782 V108 507 512 752 162 783 778 333 785 136 799 E931 157 574 568 E878 722 719 V125 296 478 V170 805 596 E880 822 733 578 459 438 008 V098 185 967 225 V457 389 412 593 345 201 515 E933 278 492 715 415 V105 535 608 E870 V058 513 709 E821 V173 824 911 913 E812 576 203 281 580 V450 216 V340 579 693 351 088 714 E849 307 421 786 E942 959 E928 588 364 V642 V025 252 283 784 611 622 289 446 729 V498 V456 795 E854 V667 155 V130 882 852 957 E815 466 792 434 342 153 E934 481 910 456 453 867 273 532 806 V422 V541 556 394 444 924 E960 514 763 218 359 340 999 451 324 E939 537 737 455 E884 V427 591 592 577 557 575 356 368 552 500 750 253 292 E937 211 288 773 314 V652 432 379 435 E930 199 V641 494 966 758 E855 741 918 V436 078 562 820 801 839 E881 V584 731 E885 812 156 567 696 501 712 V707 215 754 753 508 876 720 V442 871 958 802 847 397 196 346 E968 510 404 360 376 370 V026 904 928 821 823 150 573 850 V497 E938 V533 V556 728 870 V874 V153 V644 V600 521 301 164 054 344 464 442 V150 282 V08 891 808 866 902 117 484 760 V048 691 519 528 320 369 685 V625 794 793 318 V441 761 936 E915 457 395 053 V113 V632 386 623 290 204 271 E819 811 813 884 E813 751 366 297 V440 473 E910 V420 057 536 152 970 485 235 372 E882 127 160 170 V880 595 909 V443 490 343 319 130 698 E823 246 854 868 872 982 151 V853 980 E980 291 517 268 487 E866 796 V452 036 354 648 701 V063 V038 227 614 533 736 942 E924 240 921 V454 977 759 768 923 E816 681 138 358 950 922 205 990 009 619 417 279 257 E860 755 991 E957 241 810 920 V461 V127 261 429 550 874 756 935 831 718 962 E858 803 480 674 277 880 879 377 529 047 083 835 462 336 E947 V160 420 317 454 E883 840 V550 960 586 933 597 350 E911 742 V614 298 V551 620 716 V462 V180 706 565 452 825 322 154 040 110 605 607 461 704 713 945 052 948 323 325 934 516 039 975 971 994 666 V111 907 E929 566 603 405 049 237 V161 V553 262 743 422 337 625 757 527 309 815 V163 402 869 E912 188 590 V852 V446 E852 886 E919 183 862 875 877 890 E944 E936 V444 598 V552 226 E818 617 E958 V123 748 968 V298 465 972 E826 905 E969 744 E829 V301 388 V146 V151 887 375 334 E848 E918 284 E876 260 987 E890 834 522 692 V588 310 863 E834 192 035 V174 171 738 220 477 212 172 V548 726 526 V099 777 749 E922 952 V320 901 542 449 V011 963 E822 524 V052 V539 144 445 321 380 604 383 587 137 845 695 V496 180 618 V102 540 525 916 174 V628 892 816 V171 520 708 176 791 V854 E906 V714 V554 V435 883 927 V434 007 581 V202 140 642 644 654 V270 V252 193 V838 V555 139 V195 V068 601 826 694 626 956 245 919 299 727 684 647 E941 V850 665 391 308 633 639 V230 V061 223 269 V183 046 534 361 673 643 986 005 034 382 239 232 V169 E901 908 634 836 616 E917 734 V698 133 E887 V445 V155 E949 142 E987 236 470 463 E940 229 448 702 182 E825 V851 814 V881 259 906 161 E891 830 E953 195 093 472 914 E988 930 543 686 900 075 705 939 381 V311 V168 018 004 917 483 656 641 217 V291 V164 E943 134 635 659 E920 506 E869 111 096 094 123 158 141 243 690 097 632 989 964 027 V596 373 V017 254 932 187 353 669 V504 602 843 912 374 983 E864 031 210 114 646 077 V018 670 615 V638 V135 938 V580 680 878 E965 471 652 663 658 V272 213 032 148 V643 V148 V062 E989 E927 131 233 V040 V066 125 V503 V581 V292 V192 700 703 209 V029 208 697 E871 184 015 146 V140 V154 992 249 149 V142 844 175 V542 363 V152 V106 V688 V265 012 885 E955 V530 385 V124 V741 390 474 627 817 230 E817 V198 E862 258 V463 735 V024 V640 976 E861 V765 V023 V626 E828 V188 341 V560 798 V448 893 495 084 523 V653 953 V549 V095 V182 621 475 V425 058 306 V165 551 E831 V136 V109 256 219 221 961 985 828 671 E820 897 V840 926 V421 048 594 896 082 E986 541 145 267 683 V097 732 265 011 E801 V185 664 V620 E840 V166 V468 629 115 V587 E908 120 V708 098 V469 V694 E824 E970 121 838 832 460 013 V239 944 V189 946 118 326 E945 645 352 159 E967 V618 147 V908 941 312 624 V186 V145 661 010 E865 091 E886 649 E905 E962 V612 E959 502 V438 V222 163 947 V162 E946 V716 315 367 V540 846 717 V561 V175 842 V138 V703 V583 841 672 062 488 347 339 E841 086 V400 E985 655 974 V289 V604 V074 V728 371 190 V126 090 143 943 V611 V331 085 V172 E835 668 740 V167 V558 E851 E811 V430 837 V072 V431 302 E923 V110 E900 V562 E963 E964 V118 V624 E800 988 833 023 V020 021 003 V660 E806 313 E954 V860 660 V449 231 V602 186 E863 E874 V721 V181 651 033 V654 E804 330 610 384 E838 E001 973 819 014 132 E899 925 207 V861 E002 E030 E000 894 E873 E999 E976 E003 V016 E805 045 V610 V078 V510 E029 848 E006 V403 122 V536 E013 E019 173 E913 677 E008 V568 V143 V091 V872 066 V601 116 V882 V065 538 V655 316 E007 E016 E921 V902 206 V254 099 V489 V870 E977 628 V250 E982 V486 539 V073 937 V812 030 V271 589 V672 V671 E926 E925 E857 V537 954 E827 657 V910 V789 V037 E975 V045 V848 393 V426 179 387 V903 E856 V901 915').split(' ')


In [41]:
# from huggingface_hub import HfApi, model_info

# # Get all repository references
# api = HfApi()
# refs = api.list_repo_refs("ashishkgpian/biolink_large_disease_classification")

# refs

In [ ]:
# pip install neo4j

In [3]:

# print(classes)

from neo4j import GraphDatabase
from transformers import pipeline
import pandas as pd

# Connect to Neo4j
class Neo4jHandler:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()

    def query_kg(self, symptoms, true_length):
        """
        Query the KG for diseases related to the given symptoms.
        """
        query = """
        MATCH (s:Symptom)-[r:HAS_SYMPTOM]-(d:Disease)
    WHERE s.name IN $symptoms OR any(synonym IN s.synonyms WHERE synonym IN $symptoms)
    RETURN d.id AS disease, r.weight AS weight
    ORDER BY weight DESC
    LIMIT $true_length
        """
        with self.driver.session() as session:
            result = session.run(query, symptoms=symptoms, true_length=true_length)
            return [{"disease": record["disease"], "weight": record["weight"]} for record in result]

neo4j_handler = Neo4jHandler("bolt://localhost:8687", "neo4j", "neo4j_pass5")

/Users/ashish/Documents/mappe/weaviate_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Get predictions with RAG
1. USING MIMIC : Pass the query expanded symptoms as well as the codes
2. USING THE PUBMED KG : Pass the output codes

## 1. MIMIC Approach


In [ ]:
#  pip install py2neo dotenv


In [4]:
import os
from py2neo import Graph
from mistralai.client import MistralClient
# from mistralai.models.chat_completion import ChatMessage
from neo4j import GraphDatabase

import json


uri = "neo4j://localhost:8687" 
auth = ("neo4j", "neo4j_pass5")

driver = GraphDatabase.driver(uri, auth=auth) 
driver.verify_connectivity()


#### For checking the matching symptoms 

In [5]:
def escape_special_chars(query):
    return query.replace("'", "\\'").replace("/", "\\/")

def search_symptoms(query_strings,limit = 5):
    all_symptoms = []
    for query_string in query_strings:
        escaped_query = escape_special_chars(query_string)  # Escape the query string
        with driver.session() as session:
            result = session.run(
                f"CALL db.index.fulltext.queryNodes('symptomIndex', '{escaped_query}') "
                "YIELD node, score "
                "RETURN node.name AS symptom, score "
                "ORDER BY score DESC "
                f"LIMIT {limit}"
            )
            symptoms = [record['symptom'] for record in result]
            all_symptoms.extend(symptoms)
    return all_symptoms

# Example usage
search_queries = ['bilateral upper extremity ecchymosses', 'hypotension', 'leukocytosis', 'afib with RVR', 'acute renal failure', 'anemia', 'upper extremity ecchymoses', 'coagulopathy', 'thrombocytopenia', 'elevated white count', 'cryptogenic cirrhosis', 'chronic diastolic CHF', 'GI bleed', 'hypothyroid']
search_results = search_symptoms(search_queries)
print(search_results)

['upper extremity tenderness', 'extremity tenderness', 'extremity tremor', 'extremity swelling', 'bilateral otalgia', 'alexia with agraphia', 'cough with fever', 'fever with rash', 'pain with eye movement', 'primary headache associated with sexual activity', 'organ failure', 'renal problem', 'renal colic', 'acute retention', 'acute dyspnea', 'upper extremity tenderness', 'extremity tenderness', 'extremity tremor', 'extremity swelling', 'lower extremity paresthesias', 'diastolic rumble', 'chronic vomiting', 'chronic constipation', 'chronic vertigo', 'pain, chronic', 'gi symptoms']


In [6]:
def get_prioritized_relationships(symptom_names, weightage=5, limit=10):
    """
    Retrieves a list of diseases associated with the given symptom names, ordered by the maximum weight of the associations.
    
    Parameters:
    symptom_names (list): A list of symptom names to search for.
    weightage (int, optional): The minimum weight threshold for the associations. Defaults to 5.
    limit (int, optional): The maximum number of results to return. Defaults to 20.
    
    Returns:
    tuple: A tuple containing:
        - codes (list): A list of disease names ordered by maximum association weight.
        - all_info (dict): A dictionary mapping disease names to a list containing the disease name and the maximum association weight.
    """
    with driver.session() as session:
        result = session.run(f"""
            MATCH (s:Symptom)-[r:ASSOCIATED_WITH]->(d:Disease)
            WHERE s.name IN $symptoms AND r.weight >= {weightage}
            WITH d.title AS disease_name, collect(s.name) AS symptoms, max(r.weight) AS max_weight
            RETURN disease_name, symptoms, max_weight
            ORDER BY max_weight DESC
            LIMIT {limit}
        """, symptoms=symptom_names)
        codes = []
        all_info = {}
        for record in result:
            codes.append(record['disease_name'])
            all_info[record['disease_name']] = [record['disease_name'], record['max_weight']]
        return codes, all_info

## 2. PUBMEDKG Approach

In [7]:



def get_predictions_with_rag(input_text, symptoms, true_length, threshold=0.8,  model_only = False ):
    """
    Get predictions from the model and refine them using the knowledge graph.
    """
    # Query the KG for related diseases
    if model_only :
         
        augmented_input = f"{input_text}"
    else : 

        kg_results = neo4j_handler.query_kg(symptoms, true_length)
        augmented_input = f"{kg_results}\n{input_text}"
    
        
        

    # Tokenize and predict using the model
    tokenized_input = tokenizer(
        augmented_input,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding='max_length'
    )
    tokenized_input = {k: v.to(device) for k, v in tokenized_input.items()}
    output = model(**tokenized_input)
    predictions = torch.sigmoid(output.logits)
    predicted_labels = [model.config.id2label[_id] for _id in (predictions > threshold).nonzero()[:, 1].tolist()]
    
    
    return predicted_labels

In [8]:
def get_predictions(input_text, threshold = 0.8) : 
    """give the EHR/Symptoms as input, and 
    get the disease codes (matching in the classes under consideration) 
    as the output"""

    tokenized_input =  tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512,  # BERT's maximum sequence length
        padding='max_length'
    )
    tokenized_input = {k: v.to(device) for k, v in tokenized_input.items()}
    output = model(**tokenized_input)
    predictions = torch.sigmoid(output.logits)
    predicted_labels = [model.config.id2label[_id] for _id in (predictions > threshold).nonzero()[:, 1].tolist()]
    classes = str('403 486 582 585 425 276 710 724 458 287 285 275 583 558 327 228 338 789 790 V451 531 410 414 725 191 331 530 411 482 272 305 194 197 255 424 584 998 682 511 599 428 349 401 V100 V453 V586 041 251 E932 V300 V053 V290 571 070 250 570 572 286 518 038 280 263 995 303 244 112 881 903 955 E956 745 762 441 496 447 440 997 274 427 V104 V101 V120 V090 569 560 491 V458 433 436 493 996 416 V310 765 769 774 770 747 776 772 362 198 V103 746 766 V293 853 780 E888 730 357 430 293 443 V158 396 365 135 311 E935 721 214 437 242 600 189 304 711 800 E814 873 781 378 951 767 431 294 042 V141 V071 764 775 969 295 E950 266 779 355 553 965 E850 E853 426 804 E916 202 V502 398 707 348 787 564 V428 238 300 788 332 V107 V433 E879 861 423 E966 200 555 771 270 335 723 079 851 807 864 865 860 413 782 V108 507 512 752 162 783 778 333 785 136 799 E931 157 574 568 E878 722 719 V125 296 478 V170 805 596 E880 822 733 578 459 438 008 V098 185 967 225 V457 389 412 593 345 201 515 E933 278 492 715 415 V105 535 608 E870 V058 513 709 E821 V173 824 911 913 E812 576 203 281 580 V450 216 V340 579 693 351 088 714 E849 307 421 786 E942 959 E928 588 364 V642 V025 252 283 784 611 622 289 446 729 V498 V456 795 E854 V667 155 V130 882 852 957 E815 466 792 434 342 153 E934 481 910 456 453 867 273 532 806 V422 V541 556 394 444 924 E960 514 763 218 359 340 999 451 324 E939 537 737 455 E884 V427 591 592 577 557 575 356 368 552 500 750 253 292 E937 211 288 773 314 V652 432 379 435 E930 199 V641 494 966 758 E855 741 918 V436 078 562 820 801 839 E881 V584 731 E885 812 156 567 696 501 712 V707 215 754 753 508 876 720 V442 871 958 802 847 397 196 346 E968 510 404 360 376 370 V026 904 928 821 823 150 573 850 V497 E938 V533 V556 728 870 V874 V153 V644 V600 521 301 164 054 344 464 442 V150 282 V08 891 808 866 902 117 484 760 V048 691 519 528 320 369 685 V625 794 793 318 V441 761 936 E915 457 395 053 V113 V632 386 623 290 204 271 E819 811 813 884 E813 751 366 297 V440 473 E910 V420 057 536 152 970 485 235 372 E882 127 160 170 V880 595 909 V443 490 343 319 130 698 E823 246 854 868 872 982 151 V853 980 E980 291 517 268 487 E866 796 V452 036 354 648 701 V063 V038 227 614 533 736 942 E924 240 921 V454 977 759 768 923 E816 681 138 358 950 922 205 990 009 619 417 279 257 E860 755 991 E957 241 810 920 V461 V127 261 429 550 874 756 935 831 718 962 E858 803 480 674 277 880 879 377 529 047 083 835 462 336 E947 V160 420 317 454 E883 840 V550 960 586 933 597 350 E911 742 V614 298 V551 620 716 V462 V180 706 565 452 825 322 154 040 110 605 607 461 704 713 945 052 948 323 325 934 516 039 975 971 994 666 V111 907 E929 566 603 405 049 237 V161 V553 262 743 422 337 625 757 527 309 815 V163 402 869 E912 188 590 V852 V446 E852 886 E919 183 862 875 877 890 E944 E936 V444 598 V552 226 E818 617 E958 V123 748 968 V298 465 972 E826 905 E969 744 E829 V301 388 V146 V151 887 375 334 E848 E918 284 E876 260 987 E890 834 522 692 V588 310 863 E834 192 035 V174 171 738 220 477 212 172 V548 726 526 V099 777 749 E922 952 V320 901 542 449 V011 963 E822 524 V052 V539 144 445 321 380 604 383 587 137 845 695 V496 180 618 V102 540 525 916 174 V628 892 816 V171 520 708 176 791 V854 E906 V714 V554 V435 883 927 V434 007 581 V202 140 642 644 654 V270 V252 193 V838 V555 139 V195 V068 601 826 694 626 956 245 919 299 727 684 647 E941 V850 665 391 308 633 639 V230 V061 223 269 V183 046 534 361 673 643 986 005 034 382 239 232 V169 E901 908 634 836 616 E917 734 V698 133 E887 V445 V155 E949 142 E987 236 470 463 E940 229 448 702 182 E825 V851 814 V881 259 906 161 E891 830 E953 195 093 472 914 E988 930 543 686 900 075 705 939 381 V311 V168 018 004 917 483 656 641 217 V291 V164 E943 134 635 659 E920 506 E869 111 096 094 123 158 141 243 690 097 632 989 964 027 V596 373 V017 254 932 187 353 669 V504 602 843 912 374 983 E864 031 210 114 646 077 V018 670 615 V638 V135 938 V580 680 878 E965 471 652 663 658 V272 213 032 148 V643 V148 V062 E989 E927 131 233 V040 V066 125 V503 V581 V292 V192 700 703 209 V029 208 697 E871 184 015 146 V140 V154 992 249 149 V142 844 175 V542 363 V152 V106 V688 V265 012 885 E955 V530 385 V124 V741 390 474 627 817 230 E817 V198 E862 258 V463 735 V024 V640 976 E861 V765 V023 V626 E828 V188 341 V560 798 V448 893 495 084 523 V653 953 V549 V095 V182 621 475 V425 058 306 V165 551 E831 V136 V109 256 219 221 961 985 828 671 E820 897 V840 926 V421 048 594 896 082 E986 541 145 267 683 V097 732 265 011 E801 V185 664 V620 E840 V166 V468 629 115 V587 E908 120 V708 098 V469 V694 E824 E970 121 838 832 460 013 V239 944 V189 946 118 326 E945 645 352 159 E967 V618 147 V908 941 312 624 V186 V145 661 010 E865 091 E886 649 E905 E962 V612 E959 502 V438 V222 163 947 V162 E946 V716 315 367 V540 846 717 V561 V175 842 V138 V703 V583 841 672 062 488 347 339 E841 086 V400 E985 655 974 V289 V604 V074 V728 371 190 V126 090 143 943 V611 V331 085 V172 E835 668 740 V167 V558 E851 E811 V430 837 V072 V431 302 E923 V110 E900 V562 E963 E964 V118 V624 E800 988 833 023 V020 021 003 V660 E806 313 E954 V860 660 V449 231 V602 186 E863 E874 V721 V181 651 033 V654 E804 330 610 384 E838 E001 973 819 014 132 E899 925 207 V861 E002 E030 E000 894 E873 E999 E976 E003 V016 E805 045 V610 V078 V510 E029 848 E006 V403 122 V536 E013 E019 173 E913 677 E008 V568 V143 V091 V872 066 V601 116 V882 V065 538 V655 316 E007 E016 E921 V902 206 V254 099 V489 V870 E977 628 V250 E982 V486 539 V073 937 V812 030 V271 589 V672 V671 E926 E925 E857 V537 954 E827 657 V910 V789 V037 E975 V045 V848 393 V426 179 387 V903 E856 V901 915').split(' ')

    predicted_labels = list(set(classes).intersection(set(predicted_labels)))

    return predicted_labels

In [9]:


def create_binary_matrix(labels, classes):
    """
    Creates a binary matrix from a list of labels and a list of classes.

    Parameters:
    labels (list): A list of lists, where each inner list contains the labels for a single data point.
    classes (list): A list of class names.

    Returns:
    numpy.ndarray: A binary matrix where each row represents a data point and each column represents a class.
    """
    binary_matrix = np.zeros((len(labels), len(classes)), dtype=int)
    for i, label_list in enumerate(labels):
        for label in label_list:
            if label in classes:
                idx = classes.index(label)
                binary_matrix[i, idx] = 1
    return binary_matrix

In [10]:
# !pip install scikit-learn

In [11]:


from sklearn.metrics import roc_auc_score
import numpy as np

def roc_auc(probs, labels, multilabel=False, average='macro', multi_class='ovo'):
    if isinstance(labels, list):
        labels = np.array(labels, dtype=int)
    else:
        labels = labels.astype(int)

    # Filter relevant columns if multilabel is True
    y_score = probs
    if multilabel:
        # Identify classes with at least one positive label
        present_classes = np.any(labels == 1, axis=0)
        labels = labels[:, present_classes]
        y_score = np.array(probs)[:, present_classes]

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_true=labels, y_score=y_score, average=average, multi_class=multi_class)

    return {
        'roc_auc': roc_auc
    }


In [12]:


# Function to calculate precision, recall, and F1 score based on predicted and true disease codes
def calculate_f1(true_codes, predicted_codes):
    true_prefixes = {code[:3] for code in true_codes}
    pred_prefixes = {str(code)[:3] for code in predicted_codes}

    # True Positives (TP): Codes correctly predicted
    true_positives = len(true_prefixes & pred_prefixes)
    
    # False Positives (FP): Predicted codes that are not in true codes
    false_positives = len(pred_prefixes - true_prefixes)
    
    # False Negatives (FN): True codes that were not predicted
    false_negatives = len(true_prefixes - pred_prefixes)
    
    # Calculate Precision and Recall
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    
    # Calculate F1 Score
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
    
    return precision, recall, f1


In [13]:
# from collections import Counter

# my_list = [1, 2, 2, 3, 3, 3, 4, 4, 4, 4]
# counts = Counter(my_list)
# print(counts) 

In [2]:


import csv, ast
import pandas as pd
raw_test_df = pd.read_csv('symptoms_test.csv')
raw_test_df = raw_test_df.drop('Unnamed: 0',axis =1)

In [3]:
raw_test_df.head()

,Symptoms,Diseases,id,icd_9_desc,text,long_texts,short_texts,discharge_summary,short_codes,num_codes
0,"['bilateral upper extremity ecchymosses', 'hyp...","['cryptogenic cirrhosis', 'afib/SSS', 'fatty l...",155525,"Unspecified septicemia,Septic shock,Pneumoniti...",CHIEF COMPLAINT: L. hand hematoma\n\nPRESENT I...,"Unspecified septicemia,Septic shock,Pneumoniti...","Septicemia NOS,Septic shock,Food/vomit pneumon...",Admission Date: [**2171-8-2**] D...,"038,785,507,518,584,348,511,456,572,285,428,27...",37
1,"['altered mental status', 'GI bleed', 'hypoten...","['neurofibromatosis', 'HTN', 'hyperlipidemia',...",187452,"Streptococcal septicemia,Septic shock,Acute re...","CHIEF COMPLAINT: altered mental status, GI ble...","Streptococcal septicemia,Septic shock,Acute re...","Streptococcal septicemia,Septic shock,Acute re...",Admission Date: [**2172-9-8**] D...,"038,785,518,584,349,577,276,997,533,576,599,00...",35
2,"['abdominal pain', 'nonbloody emesis', 'hypote...","['Nonischemic dilated cardiomyopathy', 'hypert...",161002,"Acute pancreatitis,Acute kidney failure with l...",CHIEF COMPLAINT: abdominal pain\n\nPRESENT ILL...,"Acute pancreatitis,Acute kidney failure with l...","Acute pancreatitis,Ac kidny fail, tubr necr,Se...",Admission Date: [**2135-4-4**] D...,"577,584,038,995,410,785,518,567,425,070,576,27...",34
3,"['bloody diarrhea', 'abdominal pain', 'intermi...","['mesenteric ischemia', 'bowel ischemia', 'atr...",119210,"Infectious colitis, enteritis, and gastroenter...",CHIEF COMPLAINT: Mesenteric Ischemia\n\nPRESEN...,"Infectious colitis, enteritis, and gastroenter...","Infectious enteritis NOS,Chr stomach ulc w hem...",Admission Date: [**2136-2-9**] D...,"009,531,532,038,482,995,785,584,577,518,449,44...",34
4,"['abdominal pain', 'hypotension', 'decreased P...","['HTN', 'Type 2 DM', 'recent recurrent cholecy...",182497,"Unspecified septicemia,Toxic encephalopathy,Ot...",CHIEF COMPLAINT: abdominal pain\n\nPRESENT ILL...,"Unspecified septicemia,Toxic encephalopathy,Ot...","Septicemia NOS,Toxic encephalopathy,Ascites NE...",Admission Date: [**2104-5-20**] ...,"038,349,789,276,599,286,511,575,041,995,783,57...",34


In [15]:
input_ =  str(raw_test_df.iloc[0].Symptoms)+ '\n' + str(raw_test_df.iloc[0].text)

In [16]:
input_

"['bilateral upper extremity ecchymosses', 'hypotension', 'leukocytosis', 'afib with RVR', 'acute renal failure', 'anemia', 'upper extremity ecchymoses', 'coagulopathy', 'thrombocytopenia', 'elevated white count', 'cryptogenic cirrhosis', 'chronic diastolic CHF', 'GI bleed', 'hypothyroid']\nCHIEF COMPLAINT: L. hand hematoma\n\nPRESENT ILLNESS: Mr [**Known lastname 9780**] is an 84 yo M with a history of cryptogenic cirrhosis\n(c/b variceal bleed requiring [**Last Name (un) 10045**]/TIPS [**2171-7-9**]) and\nafib/SSS (s/p dual chamber PPM) who was recently admitted from\n[**Date range (1) 108604**] with UGI bleed and who was admitted this time on\n[**2171-8-2**] with bilateral upper extremity ecchymosses that occurred\nin the setting of therapeutic enoxaparin. He had some falls at\nrehab but according to his wife these occurred after he already\nhad the bruises. A trauma workup in the ED was notable mainly\nfor a hematocrit drop of 30.3 at last dishcarge -> 23.7. He had\nOB+ brown stool

In [17]:


from tqdm import tqdm

In [18]:
expanded_query = search_symptoms(['fever'], limit = 2)
expanded_query

['puerperal fever', 'intermittent fever']

In [19]:
# neo4j_handler.query_kg(['fever', 'feverish'], true_length)

In [20]:
# import os

# os.makedirs('infer_llm_22')


In [21]:
import weaviate
from weaviate.connect import ConnectionParams

def mimic_train_datasets(client, text, top_n=2) : 


   
    # Get the collection
    collection = client.collections.get("MedicalRecords_v3")

    # Perform a BM25 search
    response = collection.query.bm25(
        query=f"{text}",
        limit=top_n
    )

    results = []

    # Print results
    for item in response.objects:
        symptoms = item.properties['symptoms']
        codes = list(item.properties['short_codes'].split(','))
        icd_9_codes = [i[:3] for i in codes]
        text = item.properties['text']

        results.append((symptoms, icd_9_codes, text))
        
    
    return results
   
    



Testing the LLama 70b Model. 
Case 1 : Base Model on MIMIC IV symptoms + notes
Case 2 : Base Model + KG  on MIMIC Symptoms + notes
Case 3 : Fine tuned model (smaller) on Mimic symptoms + notes
Case 4 : Base Model + KG on MIMIC Symptoms + notes

In [22]:
mimic_4_df = pd.read_csv('mimic-iv-preprocessed-icd-symptoms.csv')
mimic_4_df.head()

,Unnamed: 0,hadm_id,SHORT_CODES,text,note_id,subject_id,charttime,CHIEF_COMPLAINT,PRESENT_ILLNESS,MEDICAL_HISTORY,MEDICATION_ADM,ALLERGIES,PHYSICAL_EXAM,FAMILY_HISTORY,SOCIAL_HISTORY,TEXT,Symptoms
0,0,29461342,"287,415,432,311,173,530,288,790,V586",Name: ___ Unit No: ___\n ...,13481293-DS-21,13481293,2182-10-05 00:00:00,"Chest/ABD pain, SOB\n \nMajor Surgical or Inva...",HISTORY OF PRESENT ILLNESS: ___ with h/o ITP p...,ITP (recently diagnosed in ___\nDepression\nFo...,The Preadmission Medication list is accurate a...,Ciprofloxacin\n \nAttending: ___.\n \nChief Co...,"Vitals: T97.9, BP119/79, HR 48, RR 18, 100/RA ...",Her father died at age ___ of myocardial infar...,___\nFamily History:\nHer father died at age _...,"CHIEF COMPLAINT: Chest/ABD pain, SOB\n \nMajor...","['Chest pain', 'Abdominal pain', 'Shortness of..."
1,1,21981089,"157,576,038,995,518,155,578,285,293,V667,427,0...",Name: ___ Unit No: ___\n \n...,11094046-DS-12,11094046,2126-05-11 00:00:00,Distal cholangiocarcinoma versus pancreatic ad...,___ y.o. M presenting with 3 wks of painless j...,"HCV, Dislipidemia, COPD, Arthritis/Gout, Type ...","Glipizide XL 5', Avodart 0.5', B12, Zantac 300...",Food Extracts\n \nAttending: ___.\n \nChief Co...,On Admission:\nVitals-98.3 70 130/70 15 10...,Noncontributory\n \nPhysical Exam:\nOn Admissi...,___\nFamily History:\nNoncontributory\n \nPhys...,CHIEF COMPLAINT: Distal cholangiocarcinoma ver...,"['painless jaundice', 'loss of appetite', 'wei..."
2,2,26760780,"780,784,724,723,331,E888,E849,722,787,782,729,...",Name: ___ Unit No: ___\n \nA...,18743501-DS-19,18743501,2132-03-30 00:00:00,"Neck pain, arm numbness, weakness",___ female with history of anxiety presenting ...,"hypothyroidism, post ablative. \nmigraines\ns/...",The Preadmission Medication list is accurate a...,Methimazole / sertraline / tramadol\n \nAttend...,ON ADMISSION:\nVitals - T: 97.9 BP: 104/65 HR:...,Diabetes\nHTN\n \nPhysical Exam:\nON ADMISSION...,___\nFamily History:\nDiabetes\nHTN\n \nPhysic...,"CHIEF COMPLAINT: Neck pain, arm numbness, weak...","['Neck pain', 'Arm numbness', 'Weakness', 'Nau..."
3,3,25559692,"786,289,401,272,296,V104,V125,V586",Name: ___ Unit No: ___\n \n...,18024139-DS-16,18024139,2166-01-13 00:00:00,chest pain\n \nMajor Surgical or Invasive Proc...,Ms. ___ is a ___ year old female who presents ...,1. Confirmed Pulmonary Embolus (per ___ E.D.) ...,Amlodipine 5mg daily \nAripiprazole 10 mg dai...,Latex / Toradol / Monistat 1 / Haldol / Biaxin...,"sat: 100% 2L, pain ___ \n___: Anxious female ...",Her grandmother died of ovarian cancer. Her fa...,___\nFamily History:\nHer grandmother died of ...,CHIEF COMPLAINT: chest pain\n \nMajor Surgical...,"['chest pain', 'left-sided chest pain radiatin..."
4,4,21887944,"435,784,338,311,300,V125,401,305,780",Name: ___ Unit No: ___\n \nAdm...,14574668-DS-3,14574668,2121-06-29 00:00:00,Acute-on-chronic HA in setting of RCVS,___ RHF recently admitted from ___ - ___ for\n...,- RCVS\n * seizure\n * SAH\n * stroke\n- Mi...,The Preadmission Medication list is accurate a...,No Known Allergies / Adverse Drug Reactions\n ...,On Admission:\nVitals: 10 98.1 90 107/66 16 99...,"Sister had a brain bleed, father had his first...",___\nFamily History:\nSister had a brain bleed...,CHIEF COMPLAINT: Acute-on-chronic HA in settin...,"['Acute-on-chronic headache', 'Nausea', 'Vomit..."


In [23]:
outputs = []

In [ ]:
for i, j in tqdm(mimic_4_df.iterrows(), total=len(mimic_4_df)):
    symptom_list = ast.literal_eval(j["Symptoms"])
    true_label = j.SHORT_CODES.split(',')
    true_length = len(true_label)
    # kg_results = neo4j_handler.query_kg(symptom_list, 22)
    #disease_codes = [i['disease'][:3] for i in kg_results]

    input_ =  '\n' +  str(j.Symptoms)+ '\n'  + str(j.text) #+ str(disease_codes) 
    output = get_icd9_codes_mistral(prompt_creation(input_), output_dir = '')
    outputs.append(output)

    processed_data = preprocess_icd9_codes(output, j.note_id)

    if i==500:
        break

   

In [ ]:
for i, j in tqdm(mimic_4_df.iterrows(), total=len(mimic_4_df)):

    symptom_list = ast.literal_eval(j["Symptoms"])
    true_label = j.SHORT_CODES.split(',')
    true_length = len(true_label)
    kg_results = neo4j_handler.query_kg(symptom_list, 22)
    disease_codes = [i['disease'][:3] for i in kg_results]

    input_ = str(disease_codes) + '\n' +  str(j.Symptoms)+ '\n'  + str(j.text) #+ str(disease_codes) 
    output = get_icd9_codes_mistral(prompt_creation(input_))
    outputs.append(output)

    processed_data = preprocess_icd9_codes(output, j.note_id, output_dir = 'infer_llm_mimic_4_kg')

    if i==500:
        break

    

   

In [160]:
import os
os.makedirs('infer_llm_mimic_4_kg')

In [60]:
# os.listdir('infer_llm_mimic_4')

In [90]:
mimic_4_df['SHORT_CODES'] = mimic_4_df['SHORT_CODES'].apply(lambda x : x.split(','))

In [100]:
mimic_4_df  = mimic_4_df.drop('Unnamed: 0',axis =1)

In [177]:
true_labels = []
pred_labels = []

for i in os.listdir('infer_llm_mimic_4_kg'):
    with open(f'infer_llm_mimic_4_kg/{i}', 'r') as f:
        note_id = i.split('icd9_codes_processed_')[1].split('.json')[0]
       
        data = json.loads(f.read())
        pred_codes = [i['code'] for i in data['icd9_codes']]


    # print(list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0])
    true_codes = list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0].split(',')

    true_labels.append(true_codes)
    pred_labels.append(pred_codes)
    




In [180]:
len(true_labels), len(pred_labels)

(350, 350)

In [ ]:
f1_scores , prec, rec = [], [], []
for true_label,pred_label in zip(true_labels, pred_labels) : 
    precision, recall, f1 = calculate_f1(true_label, pred_label)
    f1_scores.append(f1)
    prec.append(precision)
    rec.append(recall)

macro_f1_score = sum(f1_scores) / len(f1_scores)
macro_prec = sum(prec) / len(prec)
macro_rec = sum(rec) / len(rec)
print(f"Macro Precision Score: {macro_prec:.4f}")
print(f"Macro Recall Score: {macro_rec:.4f}")
print(f"Macro F1 Score: {macro_f1_score:.4f}")

### mimic-4  + rag (mimic-3-train) + llama70b




In [ ]:
os.makedirs('infer_llm_mimic_rag_5')

In [ ]:
client = weaviate.connect_to_local(
        port=9000,      # Custom HTTP port
        grpc_port=9001  # Custom gRPC port
    )

# Verify connection
print(f"Client is ready: {client.is_ready()}")
    

for i, j in tqdm(mimic_4_df.iterrows(), total=len(mimic_4_df)):
    if i<210:
        continue
    symptom_list = ast.literal_eval(j["Symptoms"])
    true_label = j.SHORT_CODES.split(',')
    true_length = len(true_label)
    text = j.TEXT
    # kg_results = neo4j_handler.query_kg(symptom_list, 22)
    #disease_codes = [i['disease'][:3] for i in kg_results]

    results  = mimic_train_datasets(client,text, top_n=5)

    input_ =  '\n' +  str(j.Symptoms)+ '\n'  + str(j.text) #+ str(disease_codes) 
    output = get_icd9_codes_mistral(prompt_creation_rag(input_, results))

    processed_data = preprocess_icd9_codes(output, j.note_id, output_dir='infer_llm_mimic_rag_5')

    if i==500:
        break

client.close()

In [39]:
true_labels = []
pred_labels = []

for i in os.listdir('infer_llm_mimic_rag_5'):
    with open(f'infer_llm_mimic_rag_5/{i}', 'r') as f:
        note_id = i.split('icd9_codes_processed_')[1].split('.json')[0]
       
        data = json.loads(f.read())
        pred_codes = [i['code'] for i in data['icd9_codes']]


    # print(list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0])
    true_codes = list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0].split(',')

    true_labels.append(true_codes)
    pred_labels.append(pred_codes)
    




In [40]:
len(true_labels), len(pred_labels)

(251, 251)

In [ ]:
f1_scores , prec, rec = [], [], []
for true_label,pred_label in zip(true_labels, pred_labels) : 
    precision, recall, f1 = calculate_f1(true_label, pred_label)
    f1_scores.append(f1)
    prec.append(precision)
    rec.append(recall)

macro_f1_score = sum(f1_scores) / len(f1_scores)
macro_prec = sum(prec) / len(prec)
macro_rec = sum(rec) / len(rec)
print(f"Macro Precision Score: {macro_prec:.4f}")
print(f"Macro Recall Score: {macro_rec:.4f}")
print(f"Macro F1 Score: {macro_f1_score:.4f}")

### mimic-4  + rag (mimic-3-train) + llama70b + KB



In [42]:
os.makedirs('infer_llm_mimic_rag_kg_5')

In [ ]:
client = weaviate.connect_to_local(
        port=9000,      # Custom HTTP port
        grpc_port=9001  # Custom gRPC port
    )

# Verify connection
print(f"Client is ready: {client.is_ready()}")
    

for i, j in tqdm(mimic_4_df.iterrows(), total=len(mimic_4_df)):
    if i<345:
        continue
    symptom_list = ast.literal_eval(j["Symptoms"])
    true_label = j.SHORT_CODES.split(',')
    true_length = len(true_label)
    text = j.TEXT
    kg_results = neo4j_handler.query_kg(symptom_list, 22)
    disease_codes = [i['disease'][:3] for i in kg_results]

    results  = mimic_train_datasets(client,text, top_n=5)

    input_ =    str(j.Symptoms)+ '\n'  + str(j.text) + '\n' + str(disease_codes) 
    output = get_icd9_codes_mistral(prompt_creation_rag(input_, results))

    processed_data = preprocess_icd9_codes(output, j.note_id, output_dir='infer_llm_mimic_rag_kg_5')

    if i==500:
        break

client.close()

In [48]:
true_labels = []
pred_labels = []

for i in os.listdir('infer_llm_mimic_rag_kg_5'):
    with open(f'infer_llm_mimic_rag_kg_5/{i}', 'r') as f:
        note_id = i.split('icd9_codes_processed_')[1].split('.json')[0]
       
        data = json.loads(f.read())
        pred_codes = [i['code'] for i in data['icd9_codes']]


    # print(list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0])
    true_codes = list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0].split(',')

    true_labels.append(true_codes)
    pred_labels.append(pred_codes)
    




In [49]:
len(true_labels), len(pred_labels)

(233, 233)

In [ ]:
en(true_labels), len(pred_labels)
f1_scores , prec, rec = [], [], []
for true_label,pred_label in zip(true_labels, pred_labels) : 
    precision, recall, f1 = calculate_f1(true_label, pred_label)
    f1_scores.append(f1)
    prec.append(precision)
    rec.append(recall)

macro_f1_score = sum(f1_scores) / len(f1_scores)
macro_prec = sum(prec) / len(prec)
macro_rec = sum(rec) / len(rec)
print(f"Macro Precision Score: {macro_prec:.4f}")
print(f"Macro Recall Score: {macro_rec:.4f}")
print(f"Macro F1 Score: {macro_f1_score:.4f}")

In [198]:
# pip install bitsandbytes


In [192]:
# pip install unsloth

In [ ]:
for i, j in tqdm(raw_test_df.iterrows(), total=len(raw_test_df)):
    symptom_list = ast.literal_eval(j["Symptoms"])
    true_label = j.short_codes.split(',')
    true_length = len(true_label)
    kg_results = neo4j_handler.query_kg(symptom_list, 22)
    disease_codes = [i['disease'][:3] for i in kg_results]

    input_ =  str(j.Symptoms)+ '\n' +str(disease_codes)  + '\n' + str(j.text)
    output = get_icd9_codes_groq(prompt_creation(input_))

    processed_data = preprocess_icd9_codes(output, j.id)

   

In [ ]:
# pip install pyarrow polars

## PUBMED RAG

In [24]:
## PUBMED BASED RELEVANT ARTICLES ON MIMIC 3 test set 
sym_mim_3 = pd.read_csv('deep/symptoms_mimic3_related_articles.csv')
sym_text_mim_3 = pd.read_csv('deep/text_with_symptoms_mimic3_related_articles.csv')

## PUBMED BASED RELEVANT ARTICLES ON MIMIC 4 test set (Symptoms as query, and Symptoms + Note as query)
sym_mim_4 = pd.read_csv('deep/symptoms_mimic4_related_articles.csv')
sym_text_mim_4 = pd.read_csv('deep/text_with_symptoms_mimic4_related_articles.csv')

In [25]:
sym_mim_3.head()

,Unnamed: 0,Symptoms,Diseases,id,icd_9_desc,text,long_texts,short_texts,discharge_summary,short_codes,num_codes,related_articles
0,8375,"['bilateral upper extremity ecchymosses', 'hyp...","['cryptogenic cirrhosis', 'afib/SSS', 'fatty l...",155525,"Unspecified septicemia,Septic shock,Pneumoniti...",CHIEF COMPLAINT: L. hand hematoma\n\nPRESENT I...,"Unspecified septicemia,Septic shock,Pneumoniti...","Septicemia NOS,Septic shock,Food/vomit pneumon...",Admission Date: [**2171-8-2**] D...,"038,785,507,518,584,348,511,456,572,285,428,27...",37,"[{'id': '23327286', 'contents': ""Envenomation ..."
1,3360,"['altered mental status', 'GI bleed', 'hypoten...","['neurofibromatosis', 'HTN', 'hyperlipidemia',...",187452,"Streptococcal septicemia,Septic shock,Acute re...","CHIEF COMPLAINT: altered mental status, GI ble...","Streptococcal septicemia,Septic shock,Acute re...","Streptococcal septicemia,Septic shock,Acute re...",Admission Date: [**2172-9-8**] D...,"038,785,518,584,349,577,276,997,533,576,599,00...",35,"[{'id': '22244603', 'contents': 'Hypermagnesem..."
2,381,"['abdominal pain', 'nonbloody emesis', 'hypote...","['Nonischemic dilated cardiomyopathy', 'hypert...",161002,"Acute pancreatitis,Acute kidney failure with l...",CHIEF COMPLAINT: abdominal pain\n\nPRESENT ILL...,"Acute pancreatitis,Acute kidney failure with l...","Acute pancreatitis,Ac kidny fail, tubr necr,Se...",Admission Date: [**2135-4-4**] D...,"577,584,038,995,410,785,518,567,425,070,576,27...",34,"[{'id': '24732098', 'contents': 'Collaborative..."
3,7632,"['bloody diarrhea', 'abdominal pain', 'intermi...","['mesenteric ischemia', 'bowel ischemia', 'atr...",119210,"Infectious colitis, enteritis, and gastroenter...",CHIEF COMPLAINT: Mesenteric Ischemia\n\nPRESEN...,"Infectious colitis, enteritis, and gastroenter...","Infectious enteritis NOS,Chr stomach ulc w hem...",Admission Date: [**2136-2-9**] D...,"009,531,532,038,482,995,785,584,577,518,449,44...",34,"[{'id': '2499181', 'contents': 'Histopathology..."
4,6713,"['abdominal pain', 'hypotension', 'decreased P...","['HTN', 'Type 2 DM', 'recent recurrent cholecy...",182497,"Unspecified septicemia,Toxic encephalopathy,Ot...",CHIEF COMPLAINT: abdominal pain\n\nPRESENT ILL...,"Unspecified septicemia,Toxic encephalopathy,Ot...","Septicemia NOS,Toxic encephalopathy,Ascites NE...",Admission Date: [**2104-5-20**] ...,"038,349,789,276,599,286,511,575,041,995,783,57...",34,"[{'id': '10714097', 'contents': 'Nonocclusive ..."


## MIMIC 3 test set

### Symptoms only

In [44]:
os.makedirs('infer_llm_pubmed_rag_5')

In [ ]:
# PUBMED BASED RELEVANT ARTICLES ON MIMIC 3 test set 

for i, j in tqdm(sym_mim_3.iterrows(), total=len(sym_mim_3)):

    symptom_list = ast.literal_eval(j["Symptoms"])
    true_label = j.short_codes.split(',')
    true_length = len(true_label)
    text = j.text
    # kg_results = neo4j_handler.query_kg(symptom_list, 22)
    #disease_codes = [i['disease'][:3] for i in kg_results]

    results  = eval(j.related_articles)[:5]

    input_ =   str(j.Symptoms)+ '\n'  + str(j.text) 
    output = get_icd9_codes_mistral(prompt_creation_rag(input_, results))

    processed_data = preprocess_icd9_codes(output, j.id, output_dir='infer_llm_pubmed_rag_5')

    if i==200:
        break




In [ ]:
raw_test_df = pd.read_csv('symptoms_test.csv')
raw_test_df = raw_test_df.drop('Unnamed: 0',axis =1)

In [77]:
sym_mim_3[sym_mim_3.id == 140525].short_codes

159    250,518,428,789,599,276,V586,E932,427,584,403,...
Name: short_codes, dtype: object

In [26]:
def get_predictions(file_path, test_set = 'mimic_4', mimic_3_df= sym_mim_3) : 

    true_labels = []
    pred_labels = []

    for i in os.listdir(f'{file_path}'):
        with open(f'{file_path}/{i}', 'r') as f:
            note_id = i.split('icd9_codes_processed_')[1].split('.json')[0]
        
            data = json.loads(f.read())
            pred_codes = list(set([i['code'] for i in data['icd9_codes']]))


        # print(list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0])
        if test_set == 'mimic_4' : 
            
            true_codes = list(mimic_4_df[mimic_4_df.note_id == note_id].SHORT_CODES)[0].split(',')
            
        else : 
 

            
            true_codes = list(mimic_3_df[mimic_3_df.id == int(note_id)].short_codes)[0].split(',')

            

        true_labels.append(true_codes)
        pred_labels.append(pred_codes)
    
    print(len(true_labels), len(pred_labels))

    f1_scores , prec, rec = [], [], []
    for true_label,pred_label in zip(true_labels, pred_labels) : 
        precision, recall, f1 = calculate_f1(true_label, pred_label)
        f1_scores.append(f1)
        prec.append(precision)
        rec.append(recall)

    macro_f1_score = sum(f1_scores) / len(f1_scores)
    macro_prec = sum(prec) / len(prec)
    macro_rec = sum(rec) / len(rec)
    print(f"Macro Precision Score: {macro_prec:.4f}")
    print(f"Macro Recall Score: {macro_rec:.4f}")
    print(f"Macro F1 Score: {macro_f1_score:.4f}")
        




In [ ]:
get_predictions('infer_llm_pubmed_rag_5', test_set = 'mimic_3')

## Symptoms + Note

In [91]:
os.makedirs('infer_llm_pubmed_rag_5_mimic_3_sym_note')

In [ ]:
## MIMIC-3 test set : PUBMED RAG

import gc

def store_preds(test_df, type_ , top_n_docs, output_dir_, test_type ) : 

    for i, j in tqdm(test_df.iterrows(), total=len(test_df)):

        symptom_list = ast.literal_eval(j["Symptoms"])

        if test_type == 'mimic_4' : 
            true_label = j.SHORT_CODES.split(',')
        else : 
            true_label = j.short_codes.split(',')

        true_length = len(true_label)
        text = j.text

        if type_  == 'kg': 

            kg_results = neo4j_handler.query_kg(symptom_list, 22)
            disease_codes = [i['disease'][:3] for i in kg_results]
            input_ =   str(j.Symptoms)+ '\n'  + str(j.text) + '\n' + str(disease_codes)
        
        else : 
            input_ =   str(j.Symptoms)+ '\n'  + str(j.text)

        results  = eval(j.related_articles)[:top_n_docs]

        
        output = get_icd9_codes_mistral(prompt_creation_rag(input_, results))

        if test_type == 'mimic_4' : 
            processed_data = preprocess_icd9_codes(output, j.note_id, output_dir=output_dir_)
        else : 
            processed_data = preprocess_icd9_codes(output, j.id, output_dir=output_dir_)

        if i==100:
            break
        gc.collect()





In [ ]:
store_preds(sym_text_mim_3, 'non_kg' , top_n_docs=5, output_dir_= 'infer_llm_pubmed_rag_5_mimic_3_sym_note')

In [ ]:
get_predictions('infer_llm_pubmed_rag_5_mimic_3_sym_note', test_set = 'mimic_3')

In [ ]:
## Now including kg as well

mimic 3 symptoms

In [ ]:
os.makedirs('infer_llm_pubmed_rag_5_mimic_3_sym_kg_included')
store_preds(sym_mim_3, 'kg' , top_n_docs=5, output_dir_= 'infer_llm_pubmed_rag_5_mimic_3_sym_kg_included')
get_predictions('infer_llm_pubmed_rag_5_mimic_3_sym_kg_included', test_set = 'mimic_3')

mimic 3 symptoms + ehr

In [ ]:
os.makedirs('infer_llm_pubmed_rag_5_mimic_3_sym_note_kg_included')
store_preds(sym_text_mim_3, 'kg' , top_n_docs=5, output_dir_= 'infer_llm_pubmed_rag_5_mimic_3_sym_note_kg_included')
get_predictions('infer_llm_pubmed_rag_5_mimic_3_sym_note_kg_included', test_set = 'mimic_3')

### Top-n = 10

In [ ]:
## sym only + non-kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_3_sym')
store_preds(sym_mim_3, 'non-kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_3_sym')
get_predictions('infer_llm_pubmed_rag_10_mimic_3_sym', test_set = 'mimic_3')

## sym+EHR  + non-kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_3_sym_note')
store_preds(sym_text_mim_3, 'non-kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_3_sym_note')
get_predictions('infer_llm_pubmed_rag_10_mimic_3_sym_note', test_set = 'mimic_3')

## Sym only + kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_3_sym_kg_included')
store_preds(sym_mim_3, 'kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_3_sym_kg_included')
get_predictions('infer_llm_pubmed_rag_10_mimic_3_sym_kg_included', test_set = 'mimic_3')


## Sym + EHR + kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_3_sym_note_kg_included')
store_preds(sym_text_mim_3, 'kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_3_sym_note_kg_included')
get_predictions('infer_llm_pubmed_rag_10_mimic_3_sym_note_kg_included', test_set = 'mimic_3')




# mimic-4 test set on pubmed RAG

### Top n = 5



In [ ]:
## sym only + non-kg

os.makedirs('infer_llm_pubmed_rag_5_mimic_4_sym',exist_ok=True)
store_preds(sym_mim_4, 'non-kg' , top_n_docs=5, output_dir_= 'infer_llm_pubmed_rag_5_mimic_4_sym', test_type='mimic_4')
get_predictions('infer_llm_pubmed_rag_5_mimic_4_sym', test_set = 'mimic_4')



In [ ]:
## sym+EHR  + non-kg

os.makedirs('infer_llm_pubmed_rag_5_mimic_4_sym_note',exist_ok=True)
store_preds(sym_text_mim_4, 'non-kg' , top_n_docs=5, output_dir_= 'infer_llm_pubmed_rag_5_mimic_4_sym_note', test_type='mimic_4')
get_predictions('infer_llm_pubmed_rag_5_mimic_4_sym_note', test_set = 'mimic_4')




In [ ]:
## Sym only + kg

os.makedirs('infer_llm_pubmed_rag_5_mimic_4_sym_kg_included')
store_preds(sym_mim_4, 'kg' , top_n_docs=5, output_dir_= 'infer_llm_pubmed_rag_5_mimic_4_sym_kg_included', test_type='mimic_4')
get_predictions('infer_llm_pubmed_rag_5_mimic_4_sym_kg_included', test_set = 'mimic_4')





In [ ]:
## Sym + EHR + kg

os.makedirs('infer_llm_pubmed_rag_5_mimic_4_sym_note_kg_included',exist_ok=True)
store_preds(sym_text_mim_4, 'kg' , top_n_docs=5, output_dir_= 'infer_llm_pubmed_rag_5_mimic_4_sym_note_kg_included', test_type='mimic_4')
get_predictions('infer_llm_pubmed_rag_5_mimic_4_sym_note_kg_included', test_set = 'mimic_4')


In [ ]:
import gc
gc.collect()

### Top n =10

In [ ]:
## sym only + non-kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_4_sym',exist_ok=True)
store_preds(sym_mim_4, 'non-kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_4_sym',  test_type='mimic_4')
get_predictions('infer_llm_pubmed_rag_10_mimic_4_sym', test_set = 'mimic_4')

In [ ]:
## sym+EHR  + non-kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_4_sym_note')
store_preds(sym_text_mim_4, 'non-kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_4_sym_note',  test_type='mimic_4')
get_predictions('infer_llm_pubmed_rag_10_mimic_4_sym_note', test_set = 'mimic_4')

In [ ]:
## Sym only + kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_4_sym_kg_included')
store_preds(sym_mim_4, 'kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_4_sym_kg_included',  test_type='mimic_4')
get_predictions('infer_llm_pubmed_rag_10_mimic_4_sym_kg_included', test_set = 'mimic_4')


In [ ]:
## Sym + EHR + kg

os.makedirs('infer_llm_pubmed_rag_10_mimic_4_sym_note_kg_included')
store_preds(sym_text_mim_4, 'kg' , top_n_docs=10, output_dir_= 'infer_llm_pubmed_rag_10_mimic_4_sym_note_kg_included',test_type='mimic_4' )
get_predictions('infer_llm_pubmed_rag_10_mimic_4_sym_note_kg_included', test_set = 'mimic_4')



In [ ]:


def get_results(approach = 'pubmed', model_only = True) : 
    true_labels = []
    predicted_labels = []
    # Wrap raw_test_df.iterrows() with tqdm
    output_file_path = "llmwithKG.csv"
    with open(output_file_path, mode='w', newline='', encoding="utf-8") as outfile:
        fieldnames = ["symptoms", "true_disease_codes", "predicted_diseases", "precision", "recall", "f1_score"]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        # Generate predictions and calculate F1 scores
        f1_scores = []
        prec = []
        rec = []
        for i, j in tqdm(raw_test_df.iterrows(), total=len(raw_test_df)):
            symptom_list = ast.literal_eval(j["Symptoms"])
            true_label = j.short_codes.split(',')
            
            if approach == 'pubmed' : 

                if model_only : 

                    pred_label = get_predictions_with_rag( str(j.Symptoms)+ '\n' + str(j.text) , symptom_list, 22, 0.0643, model_only = True)
                
                else : 

                    pred_label = get_predictions_with_rag( str(j.Symptoms)+ '\n' + str(j.text) , symptom_list,22, 0.0643, model_only = False)

                # pred_label = get_predictions(str(j.text) + ' ' + str(j.Symptoms), 0.0643745388060965)
                
                true_labels.append(true_label)
                predicted_labels.append(pred_label)

                precision, recall, f1 = calculate_f1(true_label, pred_label)
                f1_scores.append(f1)
                prec.append(precision)
                rec.append(recall)

                
                writer.writerow({
                    "symptoms": str(j.Symptoms),
                    "true_disease_codes": ", ".join(true_label),
                    "predicted_diseases": ", ".join(map(str, pred_label)),
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1
                })
            else : 
                try :     
                    expanded_query = search_symptoms(j.Symptoms[2:-2].split("', '"), limit = 5)

                    
                    # limit_ = len(true_label)
                    pred_label = get_prioritized_relationships(expanded_query,weightage=1,limit = len(true_label))[0]
                    
                    
                    input_text = str(j.text) + ' ' + str(j.Symptoms)
                    augmented_input = f"{input_text} \n{kg_results}"

                    # Tokenize and predict using the model
                    tokenized_input = tokenizer(
                        augmented_input,
                        return_tensors="pt",
                        truncation=True,
                        max_length=512,
                        padding='max_length'
                    )
                    tokenized_input = {k: v.to(device) for k, v in tokenized_input.items()}
                    output = model(**tokenized_input)
                    predictions = torch.sigmoid(output.logits)
                    pred_label = [model.config.id2label[_id] for _id in (predictions > 0.0643).nonzero()[:, 1].tolist()][:len(true_label)]
                    
                  
                    predicted_labels.append(pred_label)
                    true_labels.append(list(set(true_label)))
                    
                    precision, recall, f1 = calculate_f1(true_label, pred_label)

                    f1_scores.append(f1)
                    prec.append(precision)
                    rec.append(recall)

                    writer.writerow({
                    "symptoms": str(j.Symptoms),
                    "true_disease_codes": ", ".join(true_label),
                    "predicted_diseases": ", ".join(map(str, pred_label)),
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1
                })
                    
                except : 
                    print(j.Symptoms)
                




            

            if i ==500: 
                break

    return f1_scores, prec, rec, true_labels, predicted_labels




In [17]:
# pip install captum

In [ ]:
f1_scores_model, prec, rec, true_labels, predicted_labels= get_results(approach= 'pubmed', model_only= True)


macro_f1_score = sum(f1_scores_model) / len(f1_scores_model)
macro_prec = sum(prec) / len(prec)
macro_rec = sum(rec) / len(rec)
print(f"Macro Precision Score: {macro_prec:.4f}")
print(f"Macro Recall Score: {macro_rec:.4f}")
print(f"Macro F1 Score: {macro_f1_score:.4f}")
# print(len(f1_scores))

In [ ]:
f1_scores_rag, prec, rec, true_labels, predicted_labels= get_results(approach= 'pubmed', model_only= False)


macro_f1_score = sum(f1_scores_rag) / len(f1_scores_rag)
macro_prec = sum(prec) / len(prec)
macro_rec = sum(rec) / len(rec)
print(f"Macro Precision Score: {macro_prec:.4f}")
print(f"Macro Recall Score: {macro_rec:.4f}")
print(f"Macro F1 Score: {macro_f1_score:.4f}")
# print(len(f1_scores))

In [ ]:
def save_files (name) : 
    pd.Series(f1_scores_rag).to_csv(f'f2_f1_scores_rag_{name}.csv', index = False)
    pd.Series(f1_scores_model).to_csv(f'f2_f1_scores_model_{name}.csv', index= False)

In [154]:
save_files('betty')

In [ ]:
from scipy import stats
import numpy as np

def compare_methods(scores1, scores2, method1_name, method2_name, alpha=0.05):
    t_stat, p_value = stats.ttest_rel(scores1, scores2)
    
    print(f"Comparing {method1_name} vs {method2_name}")
    print(f"t-statistic: {t_stat:.4f}")
    print(f"p-value: {p_value:.4f}")
    
    if p_value < alpha:
        print("Statistically significant difference detected:")
        if t_stat > 0:
            print(f"{method1_name} performs significantly better")
        else:
            print(f"{method2_name} performs significantly better")
    else:
        print("No statistically significant difference between methods")

compare_methods(f1_scores_model, f1_scores_rag, 'Model', 'Pubmed + KG', alpha=0.05)

In [44]:
len(true_labels), len(predicted_labels)

(101, 101)

In [ ]:
# Define batch size
batch_size = 200 # Adjust based on memory limits

metrics_list = []

# Process batches directly from the labels, converting each batch to binary format on-the-fly
for i in tqdm(range(0, len(true_labels), batch_size), desc="Processing Batches"):
    batch_true_labels = true_labels[i:i + batch_size]
    batch_pred_labels = predicted_labels[i:i + batch_size]


    # Create binary matrices for each batch
    batch_true_matrix = create_binary_matrix(batch_true_labels, classes)
    batch_pred_matrix = create_binary_matrix(batch_pred_labels, classes)

    # Calculate metrics for the current batch
    batch_metrics = roc_auc(batch_pred_matrix, batch_true_matrix, multilabel=True)
    metrics_list.append(batch_metrics)


# Average metrics across all batches
final_metrics = {k: sum(d[k] for d in metrics_list) / len(metrics_list) for k in metrics_list[0]}
print(final_metrics)

In [ ]:
0.7565715921239807

In [159]:
f1_scores_dict = {
    'rag_betty' :[],
    'model_betty' :[],
    'rag_biobert' : [],
    'model_biobert' : [],
    'rag_biolinkbert' :[],
    'model_biolinkbert' : []
}
for i in f1_scores_dict.keys() : 
    f1_scores_dict[i] = pd.read_csv(f'f2_f1_scores_{i}.csv')['0'].to_list()
    

In [160]:
import numpy as np
import scipy.stats as stats
from itertools import combinations
import pandas as pd

def perform_t_tests(f1_scores):
    """
    Perform pairwise t-tests between all approaches.
    
    Parameters:
    f1_scores (dict): A dictionary with approach names as keys and F1 scores as values
    
    Returns:
    pandas.DataFrame: T-test results for each pair of approaches
    """
    results = []
    
    # Get all unique pairs of approaches
    approach_pairs = list(combinations(f1_scores.keys(), 2))
    
    for approach1, approach2 in approach_pairs:
        # Perform independent t-test
        t_statistic, p_value = stats.ttest_ind(
            f1_scores[approach1], 
            f1_scores[approach2]
        )
        
        result = {
            'Comparison': f'{approach1} vs {approach2}',

            't-statistic': t_statistic,
            # 'p-value': p_value,
            'Significant (α=0.05)': p_value < 0.05
        }
        results.append(result)
    
    # Create DataFrame and format numeric columns
    df = pd.DataFrame(results)
    
    # Format numeric columns
    numeric_cols = [col for col in df.columns if col not in ['Comparison', 'Significant (α=0.05)']]
    for col in numeric_cols:
        df[col] = df[col].map('{:.4f}'.format)
    
    return df



In [161]:

# Perform t-tests
t_test_results = perform_t_tests(f1_scores_dict)

# Print results
print("T-Test Results:")
t_test_results

# Optional: Save to CSV if needed
# t_test_results.to_csv('f1_score_ttest_results.csv', index=False)



T-Test Results:


,Comparison,t-statistic,Significant (α=0.05)
0,rag_betty vs model_betty,6.7577,True
1,rag_betty vs rag_biobert,2.5611,True
2,rag_betty vs model_biobert,9.3267,True
3,rag_betty vs rag_biolinkbert,-5.3433,True
4,rag_betty vs model_biolinkbert,4.0756,True
5,model_betty vs rag_biobert,-4.4552,True
6,model_betty vs model_biobert,1.9568,False
7,model_betty vs rag_biolinkbert,-11.4790,True
8,model_betty vs model_biolinkbert,-2.4608,True
9,rag_biobert vs model_biobert,6.8962,True
